In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay
from sklearn.feature_selection import mutual_info_classif


from scipy.stats import f_oneway

from itertools import combinations


### Funcion: eval_model

In [8]:
def eval_model(target, predictions, problem_type, metrics):
    """
    Evalúa un modelo de Machine Learning utilizando diferentes métricas especificadas por el usuario.

    Argumentos:
    target (tipo array): Valores reales del target.
    predictions (tipo array): Valores predichos por el modelo.
    problem_type (str): Tipo de problema, puede ser 'regression' o 'classification'.
    metrics (list): Lista de métricas a calcular y mostrar. Las métricas disponibles dependen del tipo de problema.
                     Para problemas de regresión: 'RMSE', 'MAE', 'MAPE', 'GRAPH'.
                     Para problemas de clasificación: 'ACCURACY', 'PRECISION', 'RECALL', 'CLASS_REPORT', 'MATRIX',
                                                        'MATRIX_RECALL', 'MATRIX_PRED', 'PRECISION_X', 'RECALL_X'.

    Returns:
    tuple: Una tupla con los valores de las métricas solicitadas en el orden especificado en la lista de métricas.
    """

    results = []

    if problem_type == 'regression':
        for metric in metrics:
            if metric == 'RMSE':
                rmse = np.sqrt(mean_squared_error(target, predictions))
                print(f"RMSE: {rmse}")
                results.append(rmse)
            elif metric == 'MAE':
                mae = mean_absolute_error(target, predictions)
                print(f"MAE: {mae}")
                results.append(mae)
            elif metric == 'MAPE':
                try:
                    mape = np.mean(np.abs((target - predictions) / target)) * 100
                    print(f"MAPE: {mape}")
                    results.append(mape)
                except ZeroDivisionError:
                    raise ValueError("No se puede calcular MAPE cuando hay valores de target iguales a cero.")
            elif metric == 'GRAPH':
                plt.figure(figsize=(8, 6))
                plt.scatter(target, predictions)
                plt.xlabel('Real')
                plt.ylabel('Predicción')
                plt.title('Gráfico de dispersión de Predicciones vs. Real')
                plt.show()
    elif problem_type == 'classification':
        for metric in metrics:
            if metric == 'ACCURACY':
                accuracy = accuracy_score(target, predictions)
                print(f"Accuracy: {accuracy}")
                results.append(accuracy)
            elif metric == 'PRECISION':
                precision = precision_score(target, predictions, average='macro')
                print(f"Precision: {precision}")
                results.append(precision)
            elif metric == 'RECALL':
                recall = recall_score(target, predictions, average='macro')
                print(f"Recall: {recall}")
                results.append(recall)
            elif metric == 'CLASS_REPORT':
                print("Classification Report:")
                print(classification_report(target, predictions))
            elif metric == 'MATRIX':
                print("Confusion Matrix (Absolute Values):")
                print(confusion_matrix(target, predictions))
            elif metric == 'MATRIX_RECALL':
                disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(target, predictions))
                disp.plot(normalize='true')
                plt.title('Confusion Matrix (Normalized by Recall)')
                plt.show()
            elif metric == 'MATRIX_PRED':
                disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(target, predictions))
                disp.plot(normalize='pred')
                plt.title('Confusion Matrix (Normalized by Prediction)')
                plt.show()
            elif 'PRECISION_' in metric:
                class_label = metric.split('_')[-1]
                try:
                    precision_class = precision_score(target, predictions, labels=[class_label])
                    print(f"Precision for class {class_label}: {precision_class}")
                    results.append(precision_class)
                except ValueError:
                    raise ValueError(f"La clase {class_label} no está presente en las predicciones.")
            elif 'RECALL_' in metric:
                class_label = metric.split('_')[-1]
                try:
                    recall_class = recall_score(target, predictions, labels=[class_label])
                    print(f"Recall for class {class_label}: {recall_class}")
                    results.append(recall_class)
                except ValueError:
                    raise ValueError(f"La clase {class_label} no está presente en las predicciones.")
    else:
        raise ValueError("El tipo de problema debe ser 'regression' o 'classification'.")

    return tuple(results)

*Añado una versión alternativa, donde se muestran todas las métricas posibles para cada tipo de problema de predicción*

In [9]:
def eval_model_all_metrics(target, predictions, problem_type):
    """
    Evalúa un modelo de Machine Learning utilizando todas las métricas disponibles para el tipo de problema especificado.

    Argumentos:
    target (tipo array): Valores reales del target.
    predictions (tipo array): Valores predichos por el modelo.
    problem_type (str): Tipo de problema, puede ser 'regression' o 'classification'.

    Returns:
    tuple: Una tupla con los valores de las métricas calculadas en el orden especificado.
    """

    results = []

    if problem_type == 'regression':
        rmse = np.sqrt(mean_squared_error(target, predictions))
        mae = mean_absolute_error(target, predictions)
        try:
            mape = np.mean(np.abs((target - predictions) / target)) * 100
        except ZeroDivisionError:
            raise ValueError("No se puede calcular MAPE cuando hay valores de target iguales a cero.")
        
        print(f"RMSE: {rmse}")
        print(f"MAE: {mae}")
        print(f"MAPE: {mape}")
        results.extend([rmse, mae, mape])
        
        plt.figure(figsize=(8, 6))
        plt.scatter(target, predictions)
        plt.xlabel('Real')
        plt.ylabel('Predicción')
        plt.title('Gráfico de dispersión de Predicciones vs. Real')
        plt.show()

    elif problem_type == 'classification':
        accuracy = accuracy_score(target, predictions)
        precision = precision_score(target, predictions, average='macro')
        recall = recall_score(target, predictions, average='macro')
        
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        results.extend([accuracy, precision, recall])
        
        print("Classification Report:")
        print(classification_report(target, predictions))
        
        print("Confusion Matrix (Absolute Values):")
        print(confusion_matrix(target, predictions))
        
        disp_recall = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(target, predictions))
        disp_recall.plot(normalize='true')
        plt.title('Confusion Matrix (Normalized by Recall)')
        plt.show()
        
        disp_pred = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(target, predictions))
        disp_pred.plot(normalize='pred')
        plt.title('Confusion Matrix (Normalized by Prediction)')
        plt.show()
        
        unique_labels = np.unique(target)
        for label in unique_labels:
            precision_class = precision_score(target, predictions, labels=[label])
            recall_class = recall_score(target, predictions, labels=[label])
            print(f"Precision for class {label}: {precision_class}")
            print(f"Recall for class {label}: {recall_class}")
            results.extend([precision_class, recall_class])

    else:
        raise ValueError("El tipo de problema debe ser 'regression' o 'classification'.")

    return tuple(results)

### Funcion: get_features_num_classification


In [10]:
def get_features_num_classification(dataframe, target_col, pvalue=0.05):
    """
    Devuelve una lista con las columnas numéricas del dataframe cuyo ANOVA con la columna designada por "target_col"
    supere el test de hipótesis con significación mayor o igual a 1-pvalue.

    Argumentos:
    dataframe (pd.DataFrame): DataFrame que contiene los datos.
    target_col (str): Nombre de la columna que debería ser el target de un modelo de clasificación.
    pvalue (float): Valor de significación para el test de hipótesis. Por defecto es 0.05.

    Returns:
    list: Lista de columnas numéricas cuyo ANOVA supera el test de hipótesis.
    """
    
    # Comprobación de que el dataframe no esté vacío
    if dataframe.empty:
        print("Error: El dataframe está vacío.")
        return None
    
    # Comprobación de que el target_col exista en el dataframe
    if target_col not in dataframe.columns:
        print(f"Error: La columna {target_col} no existe en el dataframe.")
        return None
    
    # Comprobación de que target_col sea categórica
    if not pd.api.types.is_categorical_dtype(dataframe[target_col]):
        print(f"Error: La columna {target_col} no es categórica.")
        return None
    
    # Filtrar columnas numéricas
    numeric_columns = dataframe.select_dtypes(include=['float64', 'int64']).columns
    
    # Comprobación de que hay al menos una columna numérica
    if not numeric_columns.any():
        print("Error: No hay columnas numéricas en el dataframe.")
        return None
    
    # Comprobación de que el pvalue sea un float válido entre 0 y 1
    if not isinstance(pvalue, float) or pvalue < 0 or pvalue > 1:
        print("Error: El valor de pvalue debe ser un float válido entre 0 y 1.")
        return None
    
    # Comprobación de que el dataframe tenga al menos dos categorías en target_col
    if len(dataframe[target_col].unique()) < 2:
        print(f"Error: La columna {target_col} tiene menos de dos categorías únicas.")
        return None
    
    # Realizar ANOVA para cada columna numérica
    selected_features = []
    for col in numeric_columns:
        _, pval = f_oneway(*[dataframe[col][dataframe[target_col] == category] for category in dataframe[target_col].unique()])
        if pval >= 1 - pvalue:
            selected_features.append(col)
    
    return selected_features

### Funcion: plot_features_num_classification


In [11]:
def plot_features_num_classification(dataframe, target_col="", columns=[], pvalue=0.05):
    """
    Genera un pairplot del dataframe considerando la columna designada por "target_col" y aquellas incluidas en "columns"
    que cumplan el test de ANOVA para el nivel 1-pvalue de significación estadística.

    Argumentos:
    dataframe (pd.DataFrame): DataFrame que contiene los datos.
    target_col (str): Nombre de la columna que se usará como variable categórica en el pairplot. Por defecto es "".
    columns (list): Lista de columnas numéricas a considerar para el pairplot. Por defecto es la lista vacía.
    pvalue (float): Valor de significación para el test de ANOVA. Por defecto es 0.05.

    Returns:
    list: Lista de columnas numéricas que cumplen con las condiciones de ANOVA.
    """

    # Comprobación de que el dataframe no esté vacío
    if dataframe.empty:
        print("Error: El dataframe está vacío.")
        return None

    # Comprobación de que target_col exista en el dataframe
    if target_col not in dataframe.columns:
        print(f"Error: La columna {target_col} no existe en el dataframe.")
        return None

    # Comprobación de que target_col sea categórica
    if not pd.api.types.is_categorical_dtype(dataframe[target_col]):
        print(f"Error: La columna {target_col} no es categórica.")
        return None

    # Filtrar columnas numéricas si columns está vacía
    if not columns:
        columns = dataframe.select_dtypes(include=['float64', 'int64']).columns.tolist()

    # Comprobación de que hay al menos una columna numérica
    if not columns:
        print("Error: No hay columnas numéricas en el dataframe.")
        return None

    # Comprobación de que el pvalue sea un float válido entre 0 y 1
    if not isinstance(pvalue, float) or pvalue < 0 or pvalue > 1:
        print("Error: El valor de pvalue debe ser un float válido entre 0 y 1.")
        return None

    # Comprobación de que el dataframe tenga al menos dos categorías en target_col
    if len(dataframe[target_col].unique()) < 2:
        print(f"Error: La columna {target_col} tiene menos de dos categorías únicas.")
        return None

    # Seleccionar solo las columnas que cumplen el test de ANOVA
    selected_columns = []
    for col in columns:
        _, pval = f_oneway(*[dataframe[col][dataframe[target_col] == category] for category in dataframe[target_col].unique()])
        if pval >= 1 - pvalue:
            selected_columns.append(col)

    # Comprobación de que hay al menos una columna seleccionada
    if not selected_columns:
        print("Error: No hay columnas que cumplan con las condiciones de ANOVA.")
        return None

    # Dividir el pairplot si el número de valores únicos en target_col es mayor que 5
    if len(dataframe[target_col].unique()) > 5:
        unique_targets = dataframe[target_col].unique()
        for i in range(0, len(unique_targets), 5):
            plot_data = dataframe[dataframe[target_col].isin(unique_targets[i:i+5])]
            plot = sns.pairplot(plot_data, hue=target_col, vars=[target_col] + selected_columns)
            plt.show()
    else:
        # Comprobar si hay que dividir el pairplot si el número de columnas seleccionadas es grande
        if len(selected_columns) > 5:
            # Dividir las columnas en grupos de máximo 5
            column_combinations = [selected_columns[i:i+4] for i in range(0, len(selected_columns), 4)]
            for cols in column_combinations:
                plot = sns.pairplot(dataframe, hue=target_col, vars=[target_col] + cols)
                plt.show()
        else:
            # Generar pairplot
            plot = sns.pairplot(dataframe, hue=target_col, vars=[target_col] + selected_columns)
            plt.show()

    return selected_columns

### Funcion: get_features_cat_classification


In [12]:
def get_features_cat_classification(dataframe, target_col, normalize=False, mi_threshold=0):
    """
    Devuelve una lista con las columnas categóricas del dataframe cuyo valor de mutual information con 'target_col'
    iguale o supere el valor de "mi_threshold".

    Argumentos:
    dataframe (pd.DataFrame): DataFrame que contiene los datos.
    target_col (str): Nombre de la columna que debería ser el target de un modelo de clasificación.
    normalize (bool): Indica si se debe normalizar el valor de mutual information. Por defecto es False.
    mi_threshold (float): Umbral de mutual information. Por defecto es 0.

    Returns:
    list: Lista de columnas categóricas que cumplen con las condiciones de mutual information.
    """

    # Comprobación de que el dataframe no esté vacío
    if dataframe.empty:
        print("Error: El dataframe está vacío.")
        return None
    
    # Comprobación de que target_col exista en el dataframe
    if target_col not in dataframe.columns:
        print(f"Error: La columna {target_col} no existe en el dataframe.")
        return None

    # Comprobación de que target_col sea categórica
    if not pd.api.types.is_categorical_dtype(dataframe[target_col]):
        print(f"Error: La columna {target_col} no es categórica.")
        return None
    
    # Comprobación de que normalize sea un booleano
    if not isinstance(normalize, bool):
        print("Error: El valor de normalize debe ser un booleano.")
        return None
    
    # Comprobación de que mi_threshold sea un float válido entre 0 y 1 si normalize es True
    if normalize and (not isinstance(mi_threshold, float) or mi_threshold < 0 or mi_threshold > 1):
        print("Error: El valor de mi_threshold debe ser un float válido entre 0 y 1 cuando normalize es True.")
        return None

    # Filtrar columnas categóricas
    categorical_columns = dataframe.select_dtypes(include=['category']).columns
    
    # Comprobación de que hay al menos una columna categórica
    if not categorical_columns.any():
        print("Error: No hay columnas categóricas en el dataframe.")
        return None

    # Calcular mutual information
    mi_values = mutual_info_classif(dataframe[categorical_columns], dataframe[target_col], discrete_features=True, random_state=42)
    
    # Normalizar si es necesario
    if normalize:
        total_mi = sum(mi_values)
        mi_values = [mi / total_mi for mi in mi_values]
    
    # Seleccionar las columnas que cumplen con el threshold
    selected_columns = [categorical_columns[i] for i, mi in enumerate(mi_values) if mi >= mi_threshold]

    return selected_columns

### Funcion: plot_features_cat_classification

In [13]:
def plot_features_cat_classification(dataframe, target_col="", columns=[], mi_threshold=0.0, normalize=False):
    """
    Pinta la distribución de etiquetas de cada valor respecto a los valores de la columna "target_col"
    para las columnas categóricas del dataframe cuyo valor de mutual information respecto de target_col
    supere el umbral dado en "mi_threshold".

    Argumentos:
    dataframe (pd.DataFrame): DataFrame que contiene los datos.
    target_col (str): Nombre de la columna que debería ser el target de un modelo de clasificación.
                       Por defecto es "".
    columns (list): Lista de columnas categóricas a considerar para pintar la distribución.
                    Por defecto es la lista vacía.
    mi_threshold (float): Umbral de mutual information. Por defecto es 0.0.
    normalize (bool): Indica si se debe normalizar el valor de mutual information. Por defecto es False.

    Returns:
    None
    """

    # Comprobación de que el dataframe no esté vacío
    if dataframe.empty:
        print("Error: El dataframe está vacío.")
        return
    
    # Comprobación de que target_col exista en el dataframe
    if target_col not in dataframe.columns:
        print(f"Error: La columna {target_col} no existe en el dataframe.")
        return

    # Comprobación de que target_col sea categórica
    if not pd.api.types.is_categorical_dtype(dataframe[target_col]):
        print(f"Error: La columna {target_col} no es categórica.")
        return
    
    # Comprobación de que mi_threshold sea un float válido entre 0 y 1 si normalize es True
    if normalize and (not isinstance(mi_threshold, float) or mi_threshold < 0 or mi_threshold > 1):
        print("Error: El valor de mi_threshold debe ser un float válido entre 0 y 1 cuando normalize es True.")
        return

    # Filtrar columnas categóricas si columns está vacía
    if not columns:
        columns = dataframe.select_dtypes(include=['category']).columns.tolist()

    # Comprobación de que hay al menos una columna categórica
    if not columns:
        print("Error: No hay columnas categóricas en el dataframe.")
        return

    # Calcular mutual information
    mi_values = mutual_info_classif(dataframe[columns], dataframe[target_col], discrete_features=True, random_state=42)
    
    # Normalizar si es necesario
    if normalize:
        total_mi = sum(mi_values)
        mi_values = [mi / total_mi for mi in mi_values]
    
    # Seleccionar las columnas que cumplen con el threshold
    selected_columns = [columns[i] for i, mi in enumerate(mi_values) if mi >= mi_threshold]

    # Comprobación de que hay al menos una columna seleccionada
    if not selected_columns:
        print("Error: No hay columnas que cumplan con las condiciones de mutual information.")
        return

    # Pintar la distribución de etiquetas para cada columna seleccionada
    for col in selected_columns:
        plt.figure(figsize=(10, 6))
        sns.countplot(x=col, hue=target_col, data=dataframe)
        plt.title(f'Distribución de {col} respecto a {target_col}')
        plt.xlabel(col)
        plt.ylabel('Conteo')
        plt.legend(title=target_col)
        plt.xticks(rotation=45)
        plt.show()